In [1]:
# Install shapely to deal with geospatial data
!pip install shapely

In [6]:
!kaggle datasets download -d dhruvildave/ookla-internet-speed-dataset -f "2021-q1/2021-01-01_performance_mobile_tiles.parquet"
!unzip 2021-01-01_performance_mobile_tiles.parquet.zip
!rm 2021-01-01_performance_mobile_tiles.parquet.zip

2021-01-01_performance_mobile_tiles.parquet.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2021-01-01_performance_mobile_tiles.parquet.zip
  inflating: 2021-01-01_performance_mobile_tiles.parquet  


In [3]:
# initlize pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline-delay-analysis").getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
22/05/13 15:20:42 WARN Utils: Your hostname, fawazalesay-pysparkairl-sdsd8mdblyh resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
22/05/13 15:20:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/13 15:20:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/13 15:20:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Needed to make Jupyter work with Gitpod
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [13]:
# Read the data into a dataframe and print the schema
df = spark.read.parquet("2021-01-01_performance_mobile_tiles.parquet")
df.printSchema()

# Print the first 5 rows of the dataframe
df.show(5)

root
 |-- quadkey: string (nullable = true)
 |-- tile: string (nullable = true)
 |-- avg_d_kbps: long (nullable = true)
 |-- avg_u_kbps: long (nullable = true)
 |-- avg_lat_ms: long (nullable = true)
 |-- tests: long (nullable = true)
 |-- devices: long (nullable = true)

+----------------+--------------------+----------+----------+----------+-----+-------+
|         quadkey|                tile|avg_d_kbps|avg_u_kbps|avg_lat_ms|tests|devices|
+----------------+--------------------+----------+----------+----------+-----+-------+
|0320102333020210|POLYGON((-76.2780...|     60057|     18502|        33|    7|      4|
|1202323320003230|POLYGON((15.52368...|     32306|      1619|        22|    2|      1|
|0231320130101122|POLYGON((-98.8989...|     40500|      8097|        39|   60|      5|
|1321231220223030|POLYGON((120.9924...|     68423|     15722|        20|  143|     52|
|1220101000310231|POLYGON((14.34265...|     31295|      7302|        37|   47|     24|
+----------------+-------------

In [32]:
from shapely import wkt

longitudes = df.select("tile").rdd.map(lambda row: wkt.loads(row.tile).centroid.x).collect()
latitudes = df.select("tile").rdd.map(lambda row: wkt.loads(row.tile).centroid.y).collect()

In [34]:
# Convert longitudes and latitudes to a dataframe
longitudes_df = spark.createDataFrame(longitudes, "double")
latitudes_df = spark.createDataFrame(latitudes, "double")

+------------------+
|             value|
+------------------+
|-76.27532958984374|
| 15.52642822265625|
|-98.89617919921875|
|120.99517822265652|
| 14.34539794921875|
+------------------+
only showing top 5 rows

None


22/05/13 15:56:04 WARN TaskSetManager: Stage 19 contains a task of very large size (7491 KiB). The maximum recommended task size is 1000 KiB.


In [38]:
print(longitudes_df.columns)

['value']


In [36]:
# Adds two columns: longitude and latitude
df = df.withColumn("longitude", longitudes_df.value)
df = df.withColumn("latitude", latitudes_df.value)

df.show(5)

AnalysisException: Resolved attribute(s) value#118 missing from quadkey#50,tile#51,avg_d_kbps#52L,avg_u_kbps#53L,avg_lat_ms#54L,tests#55L,devices#56L in operator !Project [quadkey#50, tile#51, avg_d_kbps#52L, avg_u_kbps#53L, avg_lat_ms#54L, tests#55L, devices#56L, value#118 AS longitude#128].;
!Project [quadkey#50, tile#51, avg_d_kbps#52L, avg_u_kbps#53L, avg_lat_ms#54L, tests#55L, devices#56L, value#118 AS longitude#128]
+- Relation [quadkey#50,tile#51,avg_d_kbps#52L,avg_u_kbps#53L,avg_lat_ms#54L,tests#55L,devices#56L] parquet
